# Python Microservices 

The example is taken from: https://realpython.com/python-microservices-grpc/#why-microservices

<b>Utility.</b> Microservices are a paradigm for organizing complex software systems.

<b>Principle.</b> Break your application into microservices that can be *deployed independently* and *communicate* with each other.

<b>Example.</b> Python microservices using gRPC; learn more about gRPC <a href="https://grpc.io/">here</a>.

<b>Ethos.</b> Building a framework to support critical applications, you must ensure it is <b>robust</b> and <b>developer-friendly</b>.

- <b>Robustness.</b> If all source code implements one application, then you have to deploy all code at once, which is a risky practice, i.e., small changes in the source code can take result great downtimes.

- <b>Developer-friendliness.</b> Fellow developers do not get cancer when they read your code.

## Objectives

- Implementation of microservices using Python
- Monitoring of microservices with middleware
- Unit and integration testing
- Deployment of microservices to production servers with Kubernetes

## Recommendations API

### Request object

- <b>User ID</b>: Enable personalized recommendations through machine learning.
- <b>Book category</b>: Generic book categories, no need to discover them, use the popular ones.
- <b>Max results</b>: Limit the request.

### Book object

- <b>Book ID</b>: Unique numeric ID.
- <b>Book title</b>: The title to be displayed in the user interface.

### Response object

- List of books.

### Formal definition of the API as a protocol buffer

Strictly speaking, protocol buffers refers to the serialization format of data sent between two microservices. 

In [ ]:
from IPython import display

In [ ]:
display.Code("protobufs/recommendations.proto")

<b>Remark.</b> Thnik of an `rpc` as a normal function. The difference is that the `rpc` body is executed on a remote server.

### Protocol buffers vs HTTP-JSON

- <b>Documentation.</b> Well-defined and self-documented schema.
    - Python code is generated from it, thus the code will <b>never be out of sync</b> with the documentation.
- <b>Validation.</b> If you use HTTP and JSON for your API, then you need to write code for 1) the request, 2) sending the request, 3) wait for the response, 4) check the status code, and 5) parse and validate the response. With protocol buffers, a network request is done under the hood. To get the same benefits with HTTP-JSON you can use <a href="https://swagger.io/about/">Swagger</a> to build REST APIs.
- <b>Performance.</b> gRPC is built on top of HTTP/2, which can make multiple requests in parallel on a long-lived connection in a thread-safe way. Connection setup is relatively slow, so doing it once and sharing the connection across multiple requests saves time. gRPC messages are also binary and smaller than JSON.
- <b>Developer-Friendliness.</b> Define your API in terms of functions, not HTTP verbs and resources

In [ ]:
display.Code("recommendations/run.sh")

Two Python files will be generated.

The two Python files will contain types and functions to interact with your API, i.e., client code to call an RPC and server code to implement the RPC.

The `recommendations_pb2.py` file contains the <b>type definitions</b>.

The `recommendations_pb2_grpc.py` file contains the <b>client-server framework</b>.

In [ ]:
display.Code("recommendations/recommendations_pb2.py")

In [ ]:
display.Code("recommendations/recommendations_pb2_grpc.py")

### Interacting with the compiler-generated source code

In [ ]:
import os
## Change working directory for convenience
os.chdir(os.path.join(os.getcwd(), "recommendations"))

In [ ]:
import grpc
from recommendations_pb2_grpc import RecommendationsStub
from recommendations_pb2 import BookCategory, RecommendationRequest

The <b>grpc</b> module provides functions for *setting up connections to remote servers*.

The RecommendationsStub is the RPC client stub; the client itself does not have any functionality but calls out to a remote server and passes the result back.

In [ ]:
## Make an RPC request
## Create a request for recommendation
request = RecommendationRequest(
    user_id     = 1,
    category    = BookCategory.PHILOSOPHY,
    max_results = 3
)
## If you leave one unset, then it will default to zero for numeric types or to an empty string for strings

## An insecure channel is unauthenticated and unencrypted 
## 50051 is the standard port for gRPC
channel = grpc.insecure_channel("localhost:50051") 
client  = RecommendationsStub(channel)
## Client initiates a request for recommendation; at this point there is no server
client.Recommend(request)

### RPC server implementation

In [ ]:
os.chdir(os.path.split(os.getcwd())[0])

In [ ]:
display.Code("recommendations/recommendations.py")

### Client implementation

In [ ]:
display.Code("marketplace/marketplace.py")

In [ ]:
display.Code("marketplace/templates/homepage.html")